# Getting Started with the Flow Super Agent using the AI Refinery SDK

Welcome! This notebook is a beginner-friendly guide to using the Accenture AI Refinery SDK to orchestrate a complex workflow with a **Flow Super Agent**.

This notebook uses the `DistillerClient` to manage a group of `UtilityAgent`s whose interactions are defined by a deterministic workflow. We will walk through setting up this multi-agent system to create a simple stock investment advisor.

### What is a Flow Super Agent?

A `FlowSuperAgent` is a powerful component in the AI Refinery SDK that manages other agents. Its logic is represented as a Directed Acyclic Graph (DAG), where each node is a utility agent (like a researcher or a writer) and the connections between them define the flow of information. This structure allows for complex tasks to be broken down and even processed in parallel.

### What is the AI Refinery SDK?
The AI Refinery SDK is a toolkit that makes it easier for developers to build and use powerful AI agents. It handles the complex parts of connecting to and interacting with large AI models, so you can focus on building your application. The `DistillerClient` is a key component that orchestrates how different agents work together to solve a task.

## Step 1: Installation

First, we need to install the necessary libraries. We'll install `airefinery-sdk`, `python-dotenv` for managing credentials, `PyYAML` to create our configuration, and `mermaid-py` to render the workflow diagram.

You can run the cell below by selecting it and pressing **Shift + Enter**.

In [1]:
%pip install airefinery-sdk python-dotenv PyYAML mermaid-py

Note: you may need to restart the kernel to use updated packages.


## Step 2: Securely Enter Your Credentials

To use the AI Refinery SDK, you need an **Account** and an **API Key**. These are your credentials for accessing the AI models.

**It is very important to keep your credentials private.** You should never write them directly in your code (hardcoding) or share them publicly.

We'll use Python's `getpass` library, which provides a secure way to enter your details. When you run the cell below, it will prompt you for each value in a text box that hides the characters as you type. We will then set them as environment variables, which is a common practice the SDK examples follow.

In [ ]:
import os
import getpass

# Securely get the Account and API key from the user
account = getpass.getpass("Please enter your AI Refinery Account: ")
api_key = getpass.getpass("Please enter your AI Refinery API Key: ")

# Set the credentials as environment variables for the SDK to use
os.environ["ACCOUNT"] = account
os.environ["API_KEY"] = api_key

print("Credentials have been set successfully!")

## Step 3: Create Agent Configuration and Visualize Workflow

The `DistillerClient` uses a YAML file to define all the agents and their workflows. The cell below will create the `flow_config.yaml` file for you. 

Immediately after, a second cell will read this file and render a Directed Acyclic Graph (DAG) using the `mermaid-py` library to help you visualize the agent interactions. This method generates an SVG image, ensuring it renders correctly in VS Code and other environments.

In [2]:
import yaml

config_data = {
    'orchestrator': {
        'agent_list': [{'agent_name': 'Investment Strategy Advisor'}]
    },
    'utility_agents': [
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Price Researcher', 'agent_description': 'Search stock price movements.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Trend Researcher', 'agent_description': 'Research stock market trends.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Stock Influence Researcher', 'agent_description': 'Search factors influencing stock prices.'},
        {'agent_class': 'SearchAgent', 'agent_name': 'Corporate Finance Researcher', 'agent_description': 'Research corporate finance and financial reports.'},
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Public Expectation Writer',
            'agent_description': 'Summarize public expectations based on stock trends.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'How do recent stock market trends reflect public sentiment and expectations for future price movements?', 'prompt': 'Analyze the latest stock market trends and explain how they influence public perception of future price changes.'},
                    {'question': 'What common themes emerge from stock trends that indicate investor confidence or concern?', 'prompt': 'Identify key patterns in stock trends that signal investor sentiment. Are there recurring indicators of optimism or fear?'},
                    {'question': 'Are there any major economic or geopolitical events influencing public expectations in the market?', 'prompt': 'List significant events (economic, political, or industry-specific) that have recently shaped public expectations in stock investments.'}
                ]
            }
        },
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Financial Report Writer',
            'agent_description': 'Write financial summaries based on corporate finance research.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'What are the key takeaways from the latest corporate financial reports, and how do they compare with industry benchmarks?', 'prompt': 'Summarize the most important points from the latest financial reports, including performance metrics and comparisons with industry standards.'},
                    {'question': 'How have recent earnings reports and balance sheet disclosures influenced investor sentiment?', 'prompt': 'Explain how recent earnings reports and balance sheets have impacted investor confidence, referencing key financial indicators.'},
                    {'question': 'Are there any financial indicators or metrics that stand out in the companies under research?', 'prompt': 'Highlight notable financial metrics from corporate reports that could be useful in making investment decisions.'}
                ]
            }
        },
        {
            'agent_class': 'AuthorAgent',
            'agent_name': 'Investment Suggestion Writer',
            'agent_description': 'Generate insights based on stock research and financial reports.',
            'config': {
                'memory_attribute_key': 'investment_analysis',
                'title': 'Investment Analysis Report',
                'leading_questions': [
                    {'question': 'Which stock or company is being analyzed for investment considerations?', 'prompt': 'Identify the stock or company mentioned in the conversation history. Provide context on why it is being analyzed, referencing relevant past discussions or queries.'},
                    {'question': 'What are the key insights from stock trends, financial reports, and market sentiment?', 'prompt': 'Analyze stock price movements, market trends, corporate financial reports, and investor sentiment. Identify significant patterns and factors affecting the stock’s performance.'},
                    {'question': 'Based on the analysis, should investors consider buying, selling, or holding this stock?', 'prompt': 'Evaluate the stock\'s current valuation, market trends, financial stability, and risk factors. Provide a single, definitive investment decision on whether investors should buy, sell, or hold.'}
                ]
            }
        }
    ],
    'super_agents': [
        {
            'agent_class': 'FlowSuperAgent',
            'agent_name': 'Investment Strategy Advisor',
            'agent_description': 'Provides investment insights based on stock and finance research.',
            'config': {
                'goal': 'Generate investment recommendations based on stock research, trends, financial reports, and public expectations.',
                'agent_list': [
                    {'agent_name': 'Stock Price Researcher', 'next_step': ['Stock Trend Researcher', 'Stock Influence Researcher', 'Financial Report Writer']},
                    {'agent_name': 'Stock Trend Researcher', 'next_step': ['Public Expectation Writer']},
                    {'agent_name': 'Stock Influence Researcher', 'next_step': ['Investment Suggestion Writer']},
                    {'agent_name': 'Corporate Finance Researcher', 'next_step': ['Financial Report Writer']},
                    {'agent_name': 'Financial Report Writer', 'next_step': ['Investment Suggestion Writer', 'Stock Influence Researcher']},
                    {'agent_name': 'Public Expectation Writer', 'next_step': ['Investment Suggestion Writer']},
                    {'agent_name': 'Investment Suggestion Writer'}
                ]
            }
        }
    ]
}

config_file_path = 'flow_config.yaml'
with open(config_file_path, 'w') as f:
    yaml.dump(config_data, f, default_flow_style=False)

print(f"'{config_file_path}' created successfully.")

'flow_config.yaml' created successfully.


In [4]:
import mermaid as md
from IPython.display import display, SVG

def generate_mermaid_string(config):
    """Generates a Mermaid syntax string from the agent configuration."""
    mermaid_string = "graph TD\n"
    mermaid_string += "    classDef search fill:#cde4ff,stroke:#333,stroke-width:2px;\n"
    mermaid_string += "    classDef author fill:#ffdfb3,stroke:#333,stroke-width:2px;\n"

    agent_class_map = {agent['agent_name']: agent['agent_class'] for agent in config['utility_agents']}
    # Create unique IDs for mermaid that don't contain spaces
    agent_name_to_id = {name: name.replace(' ', '') for name in agent_class_map.keys()}

    flow_agents = config['super_agents'][0]['config']['agent_list']

    # Add nodes with styles and edges
    for agent in flow_agents:
        agent_name = agent['agent_name']
        agent_id = agent_name_to_id[agent_name]
        agent_class = agent_class_map.get(agent_name)
        style_class = 'search' if agent_class == 'SearchAgent' else 'author'

        # Corrected line: removed the space after :::
        mermaid_string += f'    {agent_id}["{agent_name}"]:::{style_class}\n'

        if 'next_step' in agent:
            for next_agent_name in agent['next_step']:
                next_agent_id = agent_name_to_id[next_agent_name]
                mermaid_string += f'    {agent_id} --> {next_agent_id}\n'
    
    return mermaid_string

print("Visualizing the agent workflow by generating an SVG:")
mermaid_syntax = generate_mermaid_string(config_data)

# Use mermaid-py to generate the SVG data directly
svg_data = md.Mermaid(mermaid_syntax)
svg_data

Visualizing the agent workflow by generating an SVG:


## Step 4: Login and Initialize the Distiller Client

Now we will perform the setup required to use the client. This cell logs in with your credentials and creates a `DistillerClient` instance that is configured with our `flow_config.yaml` file. 

We only need to run this setup once. The `distiller_client` object will then be available for us to use in the following steps.

In [ ]:
from air import DistillerClient, login

# Authenticate using the environment variables we set earlier
login(
    account=str(os.getenv("ACCOUNT")),
    api_key=str(os.getenv("API_KEY")),
)
print("Login successful.")

# Create a distiller client
distiller_client = DistillerClient()

# Upload your config file to register a new distiller project
project_name = "stock_invest_advisor"
distiller_client.create_project(config_path=config_file_path, project=project_name)
print(f"Project '{project_name}' created and client is ready to use.")

## Step 5: Ask for Investment Advice

Now that the client is initialized, we can ask our first question. We will use the `distiller_client` object we created in the previous step to ask our `FlowSuperAgent` for advice.

The super agent will manage the workflow between all the researcher and writer agents to generate a comprehensive answer to our query.

**Note:** In a Jupyter environment, you can use `await` directly in a cell to run asynchronous code.

In [ ]:
from IPython.display import display, Markdown

# Use the client to ask a question to the Flow Super Agent
async with distiller_client(
    project=project_name,
    uuid="test_user_flow", # A unique identifier for the user/session
) as dc:
    print("\nSending request to the Flow Super Agent... (This may take a moment)")
    responses = await dc.query(
        query="Should I invest in NVDA this quarter?"
    )
    
    print("\n--- AI Response ---")
    async for response in responses:
        # Extract and render the 'content' field as Markdown
        display(Markdown(response["content"]))
    print("-------------------")

## Step 6: Try Your Own Questions!

Now it's your turn. You can modify the `my_question` in the cell below to ask about a different stock. Since the `distiller_client` is already set up, we can just call it again.

In [ ]:
from IPython.display import display, Markdown

# --- Try your own question here! ---
my_question = "Should I invest in MSFT this quarter?"
# ------------------------------------------------

async with distiller_client(project=project_name, uuid="test_user_flow_2") as dc:
    print(f"\nAsking the agent: '{my_question}'")
    responses = await dc.query(
        query=my_question,
    )
    print("\n--- AI Response ---")
    async for response in responses:
        # Extract and render the 'content' field as Markdown
        display(Markdown(response["content"]))
    print("-------------------")

## Congratulations!

You've successfully used the `FlowSuperAgent` to manage a complex, multi-agent workflow. 

You have learned how to:
- Install the necessary libraries.
- Securely handle your Account and API Key.
- Create a complex YAML configuration file defining multiple utility agents and a super agent.
- Visualize the agent workflow by generating an SVG image, ensuring it works in any environment.
- Initialize the `DistillerClient` once and reuse it for multiple queries.
- Render markdown output from the agents for better readability.

From here, you can explore the full documentation to learn about creating your own custom agents and workflows.